In [1]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel, GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
import torch
# import pandas as pd
from bert_utils import bert_embed_gen
from gpt_utils import generate_gpt_ans
from faiss_utils import Faiss

In [31]:
# hyperparameters to tweak
FAISS_GPT_RATIO = 0.01
THRESHOLD = 1

QUESTION_BERT_PATH = "resources/all_question_28epoch.pth"
ANSWER_BERT_PATH = "resources/all_answer_28epoch.pth"
BERT_MODEL = 'bert-base-multilingual-uncased'

GPT_PATH = "resources/model_save_3"

EMBEDDINGS_PATH = "resources/lasse_embeddings.pkl"

In [5]:
# question model
question_model = BertForMaskedLM.from_pretrained(BERT_MODEL)
question_dict = torch.load(
    QUESTION_BERT_PATH, map_location=torch.device('cpu'))
question_model.load_state_dict(question_dict)
question_model.eval()

# answer model
answer_model = BertForMaskedLM.from_pretrained(BERT_MODEL)
answer_dict = torch.load(
    ANSWER_BERT_PATH, map_location=torch.device('cpu'))
answer_model.load_state_dict(answer_dict)
answer_model.eval()

# GPT model
GPT_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium
GPT_model =  GPT2LMHeadModel.from_pretrained(GPT_PATH)
GPT_model.resize_token_embeddings(len(GPT_tokenizer))

# FAISS
faiss_obj = Faiss(EMBEDDINGS_PATH)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSe

<class 'faiss.swigfaiss_avx2.IndexFlatL2'>
True


In [23]:
import pickle5 as pickle
import faiss
from faiss import IndexIVFFlat
import numpy as np

class Faiss:
    def __init__(self, embedding_path):
        print("new version")
        with open(embedding_path,'rb') as f:
            self.embeddings = pickle.load(f)
        self.question_embedding = np.array(self.embeddings["question_embedding"])
        self.answer_embedding = np.array(self.embeddings["answer_embedding"])
        self.answers = self.embeddings["answer"]
        self.dimension = self.question_embedding[0].shape[0]
        self.nlist = 5          # number of clusters
        # self.quantiser = faiss.IndexFlatIP(self.dimension) 
        faiss.normalize_L2(self.question_embedding)  
        # print(type(self.quantiser))
        self.index_question = faiss.IndexFlatIP(self.dimension)
        self.index_question.train(self.question_embedding)  # train on the database vectors
        self.index_question.add(self.question_embedding)   # add the vectors and update the index
        print(self.index_question.is_trained) 
        self.nprobe = 2  # find 2 most similar clusters  
        self.k = 1  # return 3 nearest neighbours
    
    def get_dist_ans(self, question_emb):
        faiss.normalize_L2(question_emb)
        question_distances, question_indices = self.index_question.search(question_emb, self.k)
        print(question_distances)
        print(question_indices)
        return question_distances[0][0], self.answers[int(question_indices[0][0])]


In [29]:
# from faiss_utils import Faiss
faiss_obj = Faiss(EMBEDDINGS_PATH)

def process_question(question):
    question_length = len(question.split(" "))
    embedding = bert_embed_gen(question, question_model)
    faiss_dist, answer = faiss_obj.get_dist_ans(embedding)
    print(f"FAISS distance: {faiss_dist}")
    print(f"FAISS answer: {answer}")
    threshold_used = THRESHOLD - question_length*FAISS_GPT_RATIO
    if faiss_dist < THRESHOLD - question_length*FAISS_GPT_RATIO:
        answer = generate_gpt_ans(question, GPT_tokenizer, GPT_model)
    gpt_ans = generate_gpt_ans(question, GPT_tokenizer, GPT_model)
    print(f"GPT Ans: {gpt_ans}")
    # return str(str(answer) + " faiss:" + str(answer))

process_question("what do i do with my swollen ankle?")

new version
True


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[0.820066]]
[[7459]]
FAISS distance: 0.820065975189209
FAISS answer: well i think the first thing you need to do about your insomnia is look and see if there’s anything that you could be doing to contribute to it. is your bedroom a good environment for sleep? how much caffeine are you drinking? uh what is your stress level or anxiety level and are you doing anything to reduce that like exercise proper nutrition things of that nature. if you’ve looked at all of those areas of what i call disordered sleep and everything seems to be working pretty well i think the next step is to talk with your physician about what are some of the different things that can be done for people who have insomnia. and i don’t want people to think that the only thing you can do is take a sleeping pill for insomnia because that’s really not the case. the data is very consistent that cognitive behavioral therapy is just as effective as many of the medications on the market and actually lasts longer. cognitive b

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT Ans: the treatment will depend on the severity of your swollen ankle and which part of your ankle is swollen. if you were s assessed in the er for swelling in your ankle i would suggest that you contact the doctor's office for assistance. in some cases it is not necessary to awaken the patient to take liquid medications. in addition a phone call or emergency call to the doctor's office may be justified. if you have any other symptoms that you are experiencing including swelling in the low back scapula bruising in the joint or a yellowing of your skin (jaundice) then you should call your doctor's office for assistance.


In [34]:
def get_ans(question):
    question_length = len(question.split(" "))
    embedding = bert_embed_gen(question, question_model)
    faiss_dist, answer = faiss_obj.get_dist_ans(embedding)
    print(f"FAISS distance: {faiss_dist}")
    # print(f"FAISS answer: {answer}")
    threshold_used = THRESHOLD - question_length*FAISS_GPT_RATIO
    print(f"Threshold used: {threshold_used}")
    if faiss_dist < threshold_used:
        answer = generate_gpt_ans(question, GPT_tokenizer, GPT_model)
    return answer

get_ans("my chest is tight, what is happening to me?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[0.80013764]]
[[28530]]
FAISS distance: 0.8001376390457153
Threshold used: 0.91


'hi. such chest and chest issues are usually due to wrong sleeping position changes or changes in the medicines that you have been taking. if this isnt helping then you need to consult a cardiologist and get a detailed blood examination done. local chest x ray will help. thanks'